In [1]:
import functools

import numpy as np
import tensorflow as tf

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

16384/13049 [=====================================] - 0s 0us/step


In [3]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [6]:
head {train_file_path}

SyntaxError: invalid syntax (<ipython-input-6-a71527d3ca23>, line 1)

In [7]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [8]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [9]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [10]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'male' b'male' b'male']
age                 : [28. 21. 35.  2. 28.]
n_siblings_spouses  : [0 0 0 3 1]
parch               : [0 0 0 1 0]
fare                : [29.7   10.5   26.    21.075  7.75 ]
class               : [b'First' b'Second' b'Second' b'Third' b'Third']
deck                : [b'C' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Cherbourg' b'Southampton' b'Southampton' b'Southampton' b'Queenstown']
alone               : [b'y' b'y' b'y' b'n' b'n']


In [11]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'male' b'female' b'male' b'male' b'female']
age                 : [16. 45. 21. 35. 24.]
n_siblings_spouses  : [0 1 0 0 0]
parch               : [0 1 0 0 0]
fare                : [ 10.5   164.867  73.5   512.329  83.158]
class               : [b'Second' b'First' b'Second' b'First' b'First']
deck                : [b'unknown' b'unknown' b'unknown' b'B' b'C']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Cherbourg' b'Cherbourg']
alone               : [b'y' b'n' b'y' b'y' b'y']


In [12]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [28. 38. 30. 38. 26.]
n_siblings_spouses  : [0 1 1 0 0]
class               : [b'Third' b'First' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'C' b'unknown' b'unknown' b'unknown']
alone               : [b'y' b'n' b'n' b'y' b'y']


In [13]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [35. 22. 52.  5. 22.]
n_siblings_spouses  : [0. 1. 1. 0. 0.]
parch               : [0. 2. 1. 0. 0.]
fare                : [ 7.896 41.579 93.5   12.475  7.125]


In [14]:
example_batch, labels_batch = next(iter(temp_dataset))

In [15]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [16]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[ 28.      0.      0.      8.663]
 [ 40.      0.      0.    153.462]
 [ 18.      0.      0.     13.   ]
 [ 35.      0.      0.      7.896]
 [ 33.      0.      0.      5.   ]]

[0 1 0 0 0]


In [17]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'female' b'female' b'female']
age                 : [16. 36. 28. 28. 36.]
n_siblings_spouses  : [0 0 0 0 0]
parch               : [0 0 0 0 0]
fare                : [  9.5     0.     13.      7.75  135.633]
class               : [b'Third' b'Third' b'Second' b'Third' b'First']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'C']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Queenstown' b'Cherbourg']
alone               : [b'y' b'y' b'y' b'y' b'y']


In [18]:
example_batch, labels_batch = next(iter(temp_dataset))

In [19]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [20]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [21]:
show_batch(packed_train_data)

sex                 : [b'male' b'male' b'female' b'female' b'male']
class               : [b'Second' b'Third' b'Third' b'First' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'E' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton' b'Queenstown']
alone               : [b'y' b'y' b'n' b'n' b'y']
numeric             : [[26.     0.     0.    10.5  ]
 [19.     0.     0.     7.896]
 [29.     0.     4.    21.075]
 [33.     1.     0.    53.1  ]
 [28.     0.     0.     6.858]]


In [22]:
example_batch, labels_batch = next(iter(packed_train_data))

In [23]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [24]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [25]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

In [26]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x000001F7FA6D62F0>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [27]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[ 3.   ,  1.   ,  1.   , 18.75 ],
       [25.   ,  0.   ,  0.   ,  7.05 ],
       [30.   ,  0.   ,  0.   ,  8.663],
       [29.   ,  1.   ,  0.   , 27.721],
       [ 3.   ,  1.   ,  2.   , 41.579]], dtype=float32)>

In [28]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-2.128,  0.395,  0.782, -0.286],
       [-0.37 , -0.474, -0.479, -0.501],
       [ 0.029, -0.474, -0.479, -0.471],
       [-0.05 ,  0.395, -0.479, -0.122],
       [-2.128,  0.395,  2.043,  0.132]], dtype=float32)

In [29]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [30]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [31]:
# See what you just created.
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [32]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [33]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [34]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 0.     1.     0.     1.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.    -2.128  0.395
  0.782 -0.286  1.     0.   ]


In [35]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

In [36]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [37]:
model.fit(train_data, epochs=20)

Epoch 1/20
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
126/126 [==============================] - 1s 1ms/step - loss: 0.5352 - accuracy: 0.6999
Epoch 2/20
126/126 [==============================] - 0s 1ms/step - loss: 0.4216 - accuracy: 0.8141
Epoch 3/20
126/126 [==============================] - 0s 968us/step - loss: 0.4278 - accuracy: 0.8192
Epoch 4/20
126/126 [==============================] - 0s 1ms/step - loss: 0.4040 - accuracy: 0.8194
Epoch 5/20
126/126 [==============================] - 0s 944us/step - loss: 0.3980 - accuracy: 0.8196
Epoch 6/20
126/126 [==============================] - 0s 976us/step - loss: 0.4032 - accuracy: 0.8169
Epoch 7/20
126/126 [==============================] - 0s 944us/step - loss: 0.3912 - accuracy: 0.8134
Epoch 8/20
126/126 [==============================] - 0s 948us/step - loss: 0.3121 - accuracy: 0.8726
Epoch 9/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3775

In [38]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

Consider rewriting this model with the Functional API.
53/53 [==============================] - 0s 1ms/step - loss: 0.4555 - accuracy: 0.8447


Test Loss 0.45554330945014954, Test Accuracy 0.8446969985961914


In [39]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

Consider rewriting this model with the Functional API.
Predicted survival: 21.29%  | Actual outcome:  SURVIVED
Predicted survival: 99.91%  | Actual outcome:  SURVIVED
Predicted survival: 88.29%  | Actual outcome:  DIED
Predicted survival: 84.35%  | Actual outcome:  DIED
Predicted survival: 93.43%  | Actual outcome:  SURVIVED
